# Project 1 - Interpretability



 https://www.kaggle.com/code/jacobmillet/cs-go-round-winner-prediction-xgb-87-acc
 

## Imports

In [95]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import matplotlib as plt

## Dataset import and feature set

In [96]:
dataset = pd.read_csv("./dataset/csgo_round_snapshots.csv")
# Drop row where column 't_players_alive' has the value 6
dataset = dataset[dataset['t_players_alive'] != 6]


print(dataset.shape)


(122409, 97)


## Filter Data
We want to simplify the data to some extend in order to increase the explainability because we have 96 columns which will be too much for any layman to understand and for the model to give a clear feature-explanation especially with a narrow tree depth. Knowledge of the game is utilised. The following list contains all the modifications made
- remove columns t_score and ct_score to ensure i.i.d data. 
- group weapons according to type i.e. pistol, heavy (shotguns and machine guns), SMGs and riffles (assault riffles and sniper riffles)
- Group all explosives


In [97]:

# remove columns t_score and ct_score to ensure i.i.d data. 
dataset = dataset.drop("ct_score", axis=1)
dataset = dataset.drop("t_score", axis=1)

# group all weapons
"""
+------------+----------+-------+---------+
|   Pistol   |  Heavy   | SMGs  | Riffles |
+------------+----------+-------+---------+
|  cz75auto  |   mag7   | bizon |   ak47  |
|    glock   |   negev  | mac10 |   aug   |
| r8revolver |   nova   | mp5sd |   awp   |
|   deagle   | sawedoff |  mp7  |  g3sg1  |
| fiveseven  |  xm1014  |  mp9  | galilar |
|    tec9    |   m249   | p90   |  m4a1s  |
|    p2000   +----------+ ump45 |  m4a4   |
|    p250    |          +-------+ scar20  |
|    usps    |                  |  sg553  |
|    elite   |                  |  ssg08  |
+------------+                  |  famas  |
                                +---------+
"""
dataset['t_pistol'] = dataset['t_weapon_cz75auto'] + dataset['t_weapon_glock'] + dataset['t_weapon_r8revolver'] + dataset['t_weapon_deagle'] + dataset['t_weapon_fiveseven'] + dataset['t_weapon_tec9'] + dataset['t_weapon_p2000'] + dataset['t_weapon_p250'] + dataset['t_weapon_usps'] + dataset['t_weapon_elite']
dataset['ct_pistol'] = dataset['ct_weapon_cz75auto'] + dataset['ct_weapon_glock'] + dataset['ct_weapon_r8revolver'] + dataset['ct_weapon_deagle'] + dataset['ct_weapon_fiveseven'] + dataset['ct_weapon_tec9'] + dataset['ct_weapon_p2000'] + dataset['ct_weapon_p250'] + dataset['ct_weapon_usps'] + dataset['ct_weapon_elite']
dataset['t_heavy'] = dataset['t_weapon_mag7'] + dataset['t_weapon_negev'] + dataset['t_weapon_nova'] + dataset['t_weapon_sawedoff'] + dataset['t_weapon_xm1014'] + dataset['t_weapon_m249']
dataset['ct_heavy'] = dataset['ct_weapon_mag7'] + dataset['ct_weapon_negev'] + dataset['ct_weapon_nova'] + dataset['ct_weapon_sawedoff'] + dataset['ct_weapon_xm1014'] + dataset['ct_weapon_m249']
dataset['t_smg'] = dataset['t_weapon_bizon'] + dataset['t_weapon_mac10'] + dataset['t_weapon_mp5sd'] + dataset['t_weapon_mp7'] + dataset['t_weapon_mp9'] + dataset['t_weapon_p90'] + dataset['t_weapon_ump45']
dataset['ct_smg'] = dataset['ct_weapon_bizon'] + dataset['ct_weapon_mac10'] + dataset['ct_weapon_mp5sd'] + dataset['ct_weapon_mp7'] + dataset['ct_weapon_mp9'] + dataset['ct_weapon_p90'] + dataset['ct_weapon_ump45']
dataset['t_riffle'] = dataset['t_weapon_ak47'] + dataset['t_weapon_aug'] + dataset['t_weapon_awp'] + dataset['t_weapon_g3sg1'] + dataset['t_weapon_galilar'] + dataset['t_weapon_m4a1s'] + dataset['t_weapon_m4a4'] + dataset['t_weapon_scar20'] + dataset['t_weapon_sg553'] + dataset['t_weapon_ssg08'] + dataset['t_weapon_famas']
dataset['ct_riffle'] = dataset['ct_weapon_ak47'] + dataset['ct_weapon_aug'] + dataset['ct_weapon_awp'] + dataset['ct_weapon_g3sg1'] + dataset['ct_weapon_galilar'] + dataset['ct_weapon_m4a1s'] + dataset['ct_weapon_m4a4'] + dataset['ct_weapon_scar20'] + dataset['ct_weapon_sg553'] + dataset['ct_weapon_ssg08'] + dataset['ct_weapon_famas']
dataset = dataset.drop(dataset.columns[dataset.columns.str.contains('weapon')], axis=1)

# group all explosives
# hegrenade, flashbang, smokegrenade, incendiarygrenade, molotovgrenade, decoygrenade
dataset['t_explosives'] = dataset['t_grenade_hegrenade'] + dataset['t_grenade_flashbang'] + dataset['t_grenade_smokegrenade'] + dataset['t_grenade_incendiarygrenade'] + dataset['t_grenade_molotovgrenade']  + dataset['t_grenade_decoygrenade']
dataset['ct_explosives'] = dataset['ct_grenade_hegrenade'] + dataset['ct_grenade_flashbang'] + dataset['ct_grenade_smokegrenade'] + dataset['ct_grenade_incendiarygrenade'] + dataset['ct_grenade_molotovgrenade']  + dataset['ct_grenade_decoygrenade']
dataset = dataset.drop(dataset.columns[dataset.columns.str.contains('grenade')], axis=1)

print(dataset.shape)


(122409, 25)


# Overall Data Analysis
We have a look at some parameters about the filtered data. Viewing the X.max() we find an error in the data where 6 Ts have been counted and therefore a total health of 600 and only 4 CTs with a total health of 400. In order to ensure correct samples this sample is dropped in the beginning.

In [98]:
# print column labels
print(dataset.columns.to_list())

# print number of unique values in each column
print(dataset.nunique())

# print max and min of each column
print(dataset.max())
print(dataset.min())

print(dataset.shape)

['time_left', 'map', 'bomb_planted', 'ct_health', 't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets', 't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive', 'round_winner', 't_pistol', 'ct_pistol', 't_heavy', 'ct_heavy', 't_smg', 'ct_smg', 't_riffle', 'ct_riffle', 't_explosives', 'ct_explosives']
time_left           10782
map                     8
bomb_planted            2
ct_health             501
t_health              501
ct_armor              415
t_armor               477
ct_money             1360
t_money              1399
ct_helmets              6
t_helmets               6
ct_defuse_kits          6
ct_players_alive        6
t_players_alive         6
round_winner            2
t_pistol               11
ct_pistol              11
t_heavy                 2
ct_heavy                4
t_smg                   6
ct_smg                  6
t_riffle                6
ct_riffle               6
t_explosives           21
ct_explosives          21
dtype: int64
time

# Make ML Model
We pick a training sample space and a test sample space. We categorise the data so that the model can actually be trained on it. This includes
- split our 'map' category into categorial columns with one-hot encoding

In [99]:

# boolean CT win
dataset['CT_WIN'] = (dataset['round_winner'] == 'CT').astype('uint8')
dataset = dataset.drop(columns='round_winner')

# target and features
X = dataset.drop(columns='CT_WIN')
y = dataset['CT_WIN']

X['map'] = X['map'].astype("category")
categorical_features = X['map'].to_list()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a ColumnTransformer to apply One-Hot Encoding to the categorical feature
preprocessor = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
)

# Create a Pipeline that first applies the preprocessor and then the classifier
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.845314925251205